In [1]:
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import to_timestamp, col

In [2]:
aws_access_key_id = os.environ.get('MINIO_ACCESS_KEY')
aws_secret_key_id = os.environ.get('MINIO_SECRET_KEY')
aws_endpoint = os.environ.get('MINIO_ENDPOINT')
bucket = os.environ.get('MINIO_BUCKET')
print(aws_access_key_id, aws_access_key_id, aws_endpoint)

minio_access_key minio_access_key http://localhost:9001


In [4]:
conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
        .set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.4')
        .set('spark.hadoop.fs.s3a.fast.upload', 'true')
        .set('spark.hadoop.fs.s3a.path.style.access', 'true')
        .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')  
        .set('spark.hadoop.fs.s3a.endpoint', aws_endpoint)
        .set('spark.hadoop.fs.s3a.access.key', aws_access_key_id)
        .set('spark.hadoop.fs.s3a.secret.key', aws_secret_key_id)
        .set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
)

spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark running")

your 131072x1 screen size is bogus. expect trouble
24/06/10 20:19:36 WARN Utils: Your hostname, DESKTOP-FJ6N83N resolves to a loopback address: 127.0.1.1; using 172.20.58.135 instead (on interface eth0)
24/06/10 20:19:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/nampq/.cache/pypoetry/virtualenvs/feast-4zXjf9Ge-py3.9/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/nampq/.ivy2/cache
The jars for the packages stored in: /home/nampq/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3e3a64d2-265b-4692-b7df-94af5121e2e8;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.2.4/hadoop-aws-3.2.4.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-aws;3.2.4!hadoop-aws.jar (788ms)
:: resolution report :: resolve 12774ms :: artifacts dl 793ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlde

Spark running


In [5]:
data_id = '/home/nampq/projects/feature-store/batch_processing/test.csv'

data = spark.read.format('csv').option('header', 'true').load(data_id)

In [6]:
data.show()

+---+-------+------------------+------------------------+-------+------------------+--------------------+--------------------+--------------+----------+
|_c0|Glucose|               BMI|DiabetesPedigreeFunction|Outcome|Pregnancies_normed|BloodPressure_normed|SkinThickness_normed|Insulin_normed|Age_normed|
+---+-------+------------------+------------------------+-------+------------------+--------------------+--------------------+--------------+----------+
|  0|    161|1.2884700711337882|       1.738458570845793|      0|             0.471|               0.951|               0.606|         0.738|       0.1|
|  1|     34| 16.69615934876864|       0.992257194204867|      0|               0.0|               0.918|                0.03|         0.686|      0.55|
|  2|    109|11.949503547147923|      1.7382799665273752|      0|             0.471|               0.369|                0.01|         0.407|     0.867|
|  3|     81| 43.84995589823813|      1.6192323967547844|      1|             0.05

24/06/10 20:20:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Glucose, BMI, DiabetesPedigreeFunction, Outcome, Pregnancies_normed, BloodPressure_normed, SkinThickness_normed, Insulin_normed, Age_normed
 Schema: _c0, Glucose, BMI, DiabetesPedigreeFunction, Outcome, Pregnancies_normed, BloodPressure_normed, SkinThickness_normed, Insulin_normed, Age_normed
Expected: _c0 but found: 
CSV file: file:///home/nampq/projects/feature-store/batch_processing/test.csv


In [7]:
data \
    .write \
    .mode("overwrite") \
    .parquet(f"s3a://{bucket}/coffee_sales.parquet")

24/06/10 20:20:49 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/06/10 20:20:49 WARN FileSystem: Failed to initialize fileystem s3a://diabetes/coffee_sales.parquet: java.io.IOException: From option fs.s3a.aws.credentials.provider java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.auth.IAMInstanceCredentialsProvider not found


Py4JJavaError: An error occurred while calling o47.parquet.
: java.io.IOException: From option fs.s3a.aws.credentials.provider java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.auth.IAMInstanceCredentialsProvider not found
	at org.apache.hadoop.fs.s3a.S3AUtils.loadAWSProviderClasses(S3AUtils.java:631)
	at org.apache.hadoop.fs.s3a.S3AUtils.createAWSCredentialProviderSet(S3AUtils.java:597)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:268)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3469)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWritingFileFormat(DataSource.scala:454)
	at 